In [ ]:
!pip install anthropic
!pip install faiss-cpu
!pip install tiktoken
!pip install transformers==4.31.0
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sent

In [ ]:
import pandas as pd #library for dataframes
import tiktoken #library to estimate tokens used for each character

import faiss
# a library that allows developers to quickly search for embeddings of
# multimedia documents that are similar to each other. Can use any kind of Vector DB
# only used for demo purposes

from anthropic import Anthropic
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#never share your keys publicly. always load them using .env files. Directly assignment here is for demo purposes ONLY
ANTHROPIC_API_KEY = "XXX"
client_ant = Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

In [ ]:
embedding_encoding = "cl100k_base"
max_tokens = 5000

# load & inspect dataset
path = "/content/drive/My Drive/Colab Notebooks/AISC/Reviews.csv" #adjust to your own directory
reviews_df = pd.read_csv(path, index_col=0)

encoding = tiktoken.get_encoding(embedding_encoding) #get token usage estimate
top_n = 50 #Get 50 most recent reviews

reviews_df = reviews_df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
reviews_df = reviews_df.dropna()
reviews_df = reviews_df[:top_n]
reviews_df["combined"] = (
    "Title: " + reviews_df.Summary.str.strip() + "; Content: " + reviews_df.Text.str.strip()
)

# omit reviews that are too long to embed - OPTIONAL
reviews_df["n_tokens"] = reviews_df.combined.apply(lambda x: len(encoding.encode(x)))
reviews_df = reviews_df[reviews_df.n_tokens <= max_tokens]

In [ ]:
print(reviews_df.shape)
reviews_df.head()

(50, 8)


,Time,ProductId,UserId,Score,Summary,Text,combined,n_tokens
Id,,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...,64
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...,51
3,1219017600,B000LQOCH0,ABXLMWJIXXAIN,4,"""Delight"" says it all",This is a confection that has been around a fe...,"Title: ""Delight"" says it all; Content: This is...",137
4,1307923200,B000UA0QIQ,A395BORC6FGVXV,2,Cough Medicine,If you are looking for the secret ingredient i...,Title: Cough Medicine; Content: If you are loo...,59
5,1350777600,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy,Great taffy at a great price. There was a wid...,Title: Great taffy; Content: Great taffy at a ...,50


In [ ]:
# Initialize the SentenceTransformer model - embedding model
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_embedding_st(text):
    return sentence_model.encode(text)

reviews_df['ada_embedding'] = reviews_df.combined.apply(lambda x: get_embedding_st(x))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
reviews_df.head()

,Time,ProductId,UserId,Score,Summary,Text,combined,n_tokens,ada_embedding
Id,,,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...,64,"[-0.049665734, -0.049002055, 0.06873689, -0.00..."
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...,51,"[0.013913844, 0.06850058, 0.01119179, -0.00096..."
3,1219017600,B000LQOCH0,ABXLMWJIXXAIN,4,"""Delight"" says it all",This is a confection that has been around a fe...,"Title: ""Delight"" says it all; Content: This is...",137,"[-0.0743014, 0.0072622425, -0.043986604, 0.074..."
4,1307923200,B000UA0QIQ,A395BORC6FGVXV,2,Cough Medicine,If you are looking for the secret ingredient i...,Title: Cough Medicine; Content: If you are loo...,59,"[-0.013282064, -0.013121256, -0.08531106, 0.00..."
5,1350777600,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy,Great taffy at a great price. There was a wid...,Title: Great taffy; Content: Great taffy at a ...,50,"[-0.034084786, 0.03781792, 0.108720414, -0.007..."


In [ ]:
from sentence_transformers import util

# Load the DataFrame (if not already loaded in memory)
reviews_df["embedding"] = reviews_df["ada_embedding"].apply(np.array)

# Step 1: Create FAISS index
embedding_dim = len(reviews_df["embedding"].iloc[0])
index = faiss.IndexFlatL2(embedding_dim)  # or use IndexFlatIP for cosine similarity with normalized vectors

# Step 2: Add embeddings to the index
embeddings = np.vstack(reviews_df["embedding"].values).astype("float32")
faiss.normalize_L2(embeddings)  # normalize vectors to unit length if using cosine similarity
index.add(embeddings)


def search_reviews_st(query_text, k=5):
    """Searches reviews using SentenceTransformer embeddings."""
    query_vec = get_embedding_st(query_text)
    query_vec = query_vec.reshape(1, -1)
    faiss.normalize_L2(query_vec) # normalizse vectors to unit length if using cosine similarity

    similarities = util.cos_sim(query_vec, np.vstack(reviews_df["embedding"].values)).cpu().numpy()[0]
    indices = similarities.argsort()[-k:][::-1]

    results = reviews_df.iloc[indices][["combined", "embedding"]]
    return results


# 🔍 Example query
query = "This food tastes great and the texture is perfect"
results = search_reviews_st(query)
print(results["combined"].to_string(index=False))

Id
Title: Good Quality Dog Food; Content: I have b...
Title: Wonderful, tasty taffy; Content: This ta...
Title: good; Content: Good oatmeal.  I like the...
Title: Healthy Dog Food; Content: This is a ver...
Title: Very good but next time I won't order th...


In [ ]:
# Hitting the LLM response with relevant documents
def generate_response_with_rag(query, k=5, model="claude-3-7-sonnet-20250219"): #default model loaded in
    # Step 1: Retrieve relevant reviews
    relevant_docs = search_reviews_st(query, k=k) #rag search query
    context = "\n---\n".join(relevant_docs["combined"].tolist())

    print('\n')
    print('Relevant Documents Retreived From RAG search')
    print(context)

    # Step 2: Format the system prompt
    system_prompt = f"""
			You are a helpful assistant that answers user queries based on customer food reviews.

			Use the following context (customer reviews) to answer the question. If the context does not contain the answer, say you don't know.

			Context:
			{context}
			"""

    # Step 3: Call Claude Chat Completion API
    response = client_ant.messages.create(
        model=model,
        system = system_prompt,
        messages=[
            {"role": "user", "content": query},
            ],
        temperature=0.3,
        max_tokens=300 #set according to your needs
    )

    # Extract and return only the TextBlock
    return response.content

# 🔍 Example usage
user_query = "Are customers happy with the food's texture?" #user query
model = "claude-3-7-sonnet-20250219" #modify model version here
answer = generate_response_with_rag(user_query, model=model)

print('\n')
print("Q:", user_query)
print("A:", answer)



Relevant Documents Retreived From RAG search
Title: Good Quality Dog Food; Content: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
---
Title: Great taste and convenience; Content: We really like the McCann's steel cut oats but find we don't cook it up too often.<br />This tastes much better to me than the grocery store brands and is just as convenient.<br />Anything that keeps me eating oatmeal regularly is a good thing.
---
Title: My cats LOVE this "diet" food better than their regular food; Content: One of my boys needed to lose some weight and the other didn't.  I put this food on the floor for the chubby guy, and the protein-rich, no by-product food up higher where only my skinny boy can jump.  The higher food sits going stale.  They both really go for this food.  An